# ⚙️ Section 7: Vectorization, Universal Functions (UFuncs), and Custom UFuncs

NumPy’s speed comes from **vectorization** — replacing slow Python loops with fast, compiled operations on entire arrays. These are powered by **universal functions (ufuncs)**, which are written in C and operate element-wise on NumPy arrays.

In this section, you'll learn how to:
- Use built-in ufuncs for elementwise operations.
- Combine ufuncs to write fast, expressive code.
- Create **custom ufuncs** using `np.frompyfunc` and `np.vectorize`.
- Compare vectorized vs. loop-based performance.
- Apply vectorization in real-world contexts like **signal processing** and **financial analytics**.

## 🚀 1. Built-in UFuncs: The Power Behind Vectorization

Let’s start with a simple numerical example. Suppose we have a time series of stock returns and we want to compute compounded growth over time.

Without vectorization, you'd loop through each day — but NumPy’s ufuncs let you do this in one line.

In [ ]:
import numpy as np
import time

# Simulated daily returns for a stock (in %)
returns = np.random.normal(loc=0.1, scale=1.0, size=1_000_000) / 100

# Using ufuncs for elementwise operations
growth_factor = 1 + returns
final_value = np.prod(growth_factor)  # Compound multiplication
print(f"Final growth factor: {final_value:.3f}x")

`np.prod`, `np.exp`, `np.log`, `np.sqrt`, and many others are **ufuncs** — they operate directly on whole arrays, internally running C loops optimized with SIMD instructions.

## 🧠 2. Broadcasting and Composition of UFuncs

UFuncs support **broadcasting** automatically. This allows elementwise operations between arrays of different shapes. You can also compose multiple ufuncs together for compact, readable math.

Let’s compute the **distance matrix** between a set of 2D points.

In [ ]:
# Generate 5 random 2D points
points = np.random.rand(5, 2)

# Compute pairwise Euclidean distance using ufuncs
diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
distances = np.sqrt(np.sum(diff ** 2, axis=-1))

print("Pairwise distance matrix:\n", np.round(distances, 3))

This is pure vectorization — no explicit loops, just **broadcasting** and **ufunc composition**. NumPy internally optimizes the operations for performance close to C.

## 🧩 3. Creating Your Own UFuncs

You can build **custom ufuncs** for operations not available in NumPy. These allow you to use Python functions as vectorized operations over arrays.

### Example: A custom nonlinear transformation used in signal compression.

In [ ]:
# Define a simple nonlinear compression function
def compress(x, alpha=0.7):
    return np.sign(x) * (np.abs(x) ** alpha)

# Vectorize it
compress_vec = np.vectorize(compress)

# Test on a signal
signal = np.linspace(-1, 1, 10)
compressed_signal = compress_vec(signal)
print("Original:", np.round(signal, 3))
print("Compressed:", np.round(compressed_signal, 3))

`np.vectorize` doesn’t make your code faster — it’s syntactic sugar that applies your Python function elementwise. But it’s extremely useful for prototyping or readability.

### A truly efficient approach — `np.frompyfunc`
If you want full ufunc behavior (e.g., broadcasting and type flexibility), use `np.frompyfunc`.

In [ ]:
# Create a ufunc from a Python function
def gain(signal, factor):
    return signal * factor if abs(factor) <= 2 else signal * 2

gain_ufunc = np.frompyfunc(gain, 2, 1)  # 2 inputs, 1 output

# Apply to arrays
x = np.array([0.5, 1.0, 2.5])
factors = np.array([1.5, 3.0, 0.8])

print("Input:", x)
print("Factors:", factors)
print("Result (ufunc):", gain_ufunc(x, factors))

Unlike `np.vectorize`, `frompyfunc` returns a **true ufunc object** with full broadcasting support, though it still executes in Python space (not compiled). For true speed, you’d move to Cython or Numba later.

## 💰 4. Real-World Example: Portfolio Returns Simulation

Let’s apply ufuncs to a realistic scenario — computing portfolio returns over multiple assets and days using vectorized math.

In [ ]:
# 1000 days of returns for 4 assets
returns = np.random.normal(loc=0.001, scale=0.02, size=(1000, 4))

# Portfolio weights (must sum to 1)
weights = np.array([0.4, 0.3, 0.2, 0.1])

# Daily portfolio returns via broadcasting + ufuncs
portfolio_returns = np.sum(returns * weights, axis=1)

# Compute cumulative growth using vectorized np.cumprod
growth = np.cumprod(1 + portfolio_returns)

print(f"Final portfolio growth: {growth[-1]:.2f}x")

This is the vectorized version of what would otherwise be thousands of nested Python loops — concise, clear, and orders of magnitude faster.

## ⏱️ 5. Comparing Vectorized vs. Loop Performance

Let’s time a simple task — squaring one million numbers — using both approaches.

In [ ]:
x = np.arange(1_000_000, dtype=float)

start = time.time()
y_loop = np.array([i ** 2 for i in x])
loop_time = time.time() - start

start = time.time()
y_vec = x ** 2  # Vectorized
vec_time = time.time() - start

print(f"Loop time: {loop_time:.4f}s")
print(f"Vectorized time: {vec_time:.6f}s (~{loop_time/vec_time:.0f}x faster)")

Even simple elementwise operations can be **50–200× faster** when vectorized — because NumPy executes compiled loops in C, minimizing Python overhead.

## 🧬 Under the Hood: How UFuncs Work

- UFuncs are implemented in C and use **SIMD vectorized instructions** (like SSE/AVX) for fast math.
- They operate **elementwise**, but in **batches** — minimizing Python function calls.
- Many ufuncs support **out parameters** (`out=`) to avoid memory allocation.
- They’re also **thread-safe** and can be run across multiple cores (with OpenBLAS/MKL).
- Custom ufuncs can be JIT-compiled with **Numba** for near-native speed.

## ⚙️ Best Practices & Pitfalls

✅ Prefer built-in ufuncs whenever possible — they’re faster and memory-efficient.
✅ Chain ufuncs together; NumPy fuses operations efficiently.
✅ Use the `out=` parameter for large arrays to avoid extra allocations.
⚠️ `np.vectorize` adds readability, not speed — it still loops in Python.
⚠️ Watch for intermediate array creation in long expressions — it can increase memory use.
⚠️ For ultimate speed, combine ufuncs with **Numba** or **Cython**.

## 💪 Challenge Exercise

**Task:** You’re modeling sound intensity through an attenuation filter.
1. Generate a time array `t` of 10,000 points from 0 to 1 second.
2. Simulate a sine wave: `signal = sin(2π·440·t)`.
3. Define and vectorize a custom attenuation function: `attenuate(x, k) = x * exp(-k * |x|)`.
4. Apply it to the signal with `k = 3.5`.
5. Plot both signals (if using matplotlib) and compare amplitudes.

*Hint:* Use `np.vectorize` for simplicity, or `np.frompyfunc` for broadcasting behavior.

# --- End of Section 7 ---

Next up → **Section 8: Memory Mapping, Shared Arrays, and Performance Profiling**

You’ll explore how to handle huge datasets efficiently using memory-mapped files and shared memory arrays — essential for scalable data pipelines.